In [352]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import unidecode

In [353]:
r = requests.get('https://mundoeducacao.uol.com.br/geografia/estados-brasil.htm')
soup = BeautifulSoup(r.text,'html.parser')

In [354]:
depara_estados = pd.read_html('https://mundoeducacao.uol.com.br/geografia/estados-brasil.htm')[0]
new_header = depara_estados.iloc[0]
depara_estados = depara_estados[1:]
depara_estados.columns = new_header
depara_estados['Estado'] = depara_estados['Estado'].apply(lambda x: unidecode.unidecode(x.lower()).replace(' ','-'))
depara_estados['Sigla'] = depara_estados['Sigla'].apply(lambda x: x.lower())

In [355]:
votos = []
nomes = []
partido = []
votos_totais = []
votos_validos = []
links = []
estado = []
resultados = {}

for c in range(1,28):
    links.append('https://g1.globo.com/{}/{}/eleicoes/2022/apuracao/governador.ghtml'.format(depara_estados.loc[c,'Sigla'],depara_estados.loc[c,'Estado']))

for link in links:
    r = requests.get(link)
    soup = BeautifulSoup(r.text,'html.parser')
    qtd_candidatos = len(soup.find('ol','apuracao-executivo-candidatos js__executivo--candidatos').findAll('li'))
    for c in range(qtd_candidatos):
        votos.append(soup.find('ol','apuracao-executivo-candidatos js__executivo--candidatos').findAll('li')[c].find('p',class_='eui-update-status').text)
        nomes.append(soup.find('ol','apuracao-executivo-candidatos js__executivo--candidatos').findAll('li')[c].find('p',class_='apuracao-executivo-candidatos-item__nome eui-candidate-name').text)
        partido.append(soup.find('ol','apuracao-executivo-candidatos js__executivo--candidatos').findAll('li')[c].find('span',class_='apuracao-executivo-candidatos-item__party eui-party-name').text)
        votos_totais.append(soup.findAll('div',class_='eui-statistic-item')[0].text.replace('TOTAL','').replace(' ',''))
        votos_validos.append(soup.findAll('div',class_='eui-statistic-item')[1].text.replace('VÁLIDOS','').replace(' ','').split('(')[0])
        estado.append(link.split('/')[3])
        
    resultados['nomes'] = nomes
    resultados['votos'] = votos
    resultados['partido'] = partido
    resultados['votos_totais'] = votos_totais
    resultados['votos_validos'] = votos_validos
    resultados['estado'] = estado
    
    
resultados_2022 = pd.DataFrame(resultados)
resultados_2022['nomes'] = resultados_2022['nomes'].apply(lambda x: re.match(' (.+) ',x).group(1))
resultados_2022['partido'] = resultados_2022['partido'].apply(lambda x: re.match(' (.+) ',x).group(1))
resultados_2022['votos_totais'] = resultados_2022['votos_totais'].apply(lambda x: x.replace('.',''))
resultados_2022['votos_validos'] = resultados_2022['votos_totais'].apply(lambda x: x.replace('.',''))
resultados_2022['votos'] = resultados_2022['votos'].apply(lambda x: (re.match(' (.+) ',x).group(1)).replace('votos','').replace(' ','').replace('.',''))
resultados_2022['turno'] = 1
resultados_2022['ano'] = 2022

In [356]:
votos = []
nomes = []
partido = []
votos_totais = []
votos_validos = []
turno_lst = []
links = []
estado = []
resultados = {}

for c in range(1,28):
    links.append('https://g1.globo.com/politica/eleicoes/2018/{}/apuracao/governador.ghtml'.format(depara_estados.loc[c,'Sigla']))

for link in links:
    r = requests.get(link)
    soup = BeautifulSoup(r.text,'html.parser')
    
    turnos = []
    turno1 = soup.find('div',attrs={'id':'turno1'})
    turno2 = soup.find('div',attrs={'id':'turno2'})
    
    if turno2 != None:
        turnos = [turno1,turno2]
    else:
        turnos = [turno1]
    
    for n_turno,turno in enumerate(turnos):
        qtd_candidatos = len(turno.find('ol','apuracao-executivo-candidatos js__executivo--candidatos').findAll('li'))
        for c in range(qtd_candidatos):
            votos.append(turno.find('ol','apuracao-executivo-candidatos js__executivo--candidatos').findAll('li')[c].find('p',class_='eui-update-status').text)
            nomes.append(turno.find('ol','apuracao-executivo-candidatos js__executivo--candidatos').findAll('li')[c].find('p',class_='apuracao-executivo-candidatos-item__nome eui-candidate-name').text)
            partido.append(turno.find('ol','apuracao-executivo-candidatos js__executivo--candidatos').findAll('li')[c].find('span',class_='apuracao-executivo-candidatos-item__party eui-party-name').text)
            votos_totais.append(turno.findAll('div',class_='eui-statistic-item')[0].text.replace('TOTAL','').replace(' ',''))
            votos_validos.append(turno.findAll('div',class_='eui-statistic-item')[1].text.replace('VÁLIDOS','').replace(' ','').split('(')[0])
            turno_lst.append(n_turno+1)
            estado.append(link.split('/')[6])
    resultados['nomes'] = nomes
    resultados['votos'] = votos
    resultados['partido'] = partido
    resultados['votos_totais'] = votos_totais
    resultados['votos_validos'] = votos_validos
    resultados['turno'] = turno_lst
    resultados['estado'] = estado
    
resultados_2018 = pd.DataFrame(resultados)
resultados_2018['nomes'] = resultados_2018['nomes'].apply(lambda x: re.match(' (.+) ',x).group(1))
resultados_2018['partido'] = resultados_2018['partido'].apply(lambda x: re.match(' (.+) ',x).group(1))
resultados_2018['votos_totais'] = resultados_2018['votos_totais'].apply(lambda x: x.replace('.',''))
resultados_2018['votos_validos'] = resultados_2018['votos_totais'].apply(lambda x: x.replace('.',''))
resultados_2018['votos'] = resultados_2018['votos'].apply(lambda x: (re.match(' (.+) ',x).group(1)).replace('votos','').replace(' ','').replace('.',''))
resultados_2018['ano'] = 2018

# Deu problema na coleta do 2 turno
turno2_2018 = pd.read_csv('turno2_2018 - turno2.csv')
turno2_2018.drop('index',1,inplace=True)
turno2_2018['votos'] = turno2_2018['votos'].apply(lambda x: x.replace('.',''))
turno2_2018['votos_totais'] = turno2_2018['votos_totais'].apply(lambda x: x.replace('.',''))
turno2_2018['votos_validos'] = turno2_2018['votos_validos'].apply(lambda x: x.replace('.',''))

turno1_2018 = resultados_2018[resultados_2018['turno']==1]
resultados_2018 = pd.concat([turno1_2018,turno2_2018])

/var/folders/ts/qnrqfkqx7ldcn69lydmzhd2h0000gn/T/ipykernel_21550/43435602.py:55: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  turno2_2018.drop('index',1,inplace=True)


In [392]:
resultados = pd.concat([resultados_2018,resultados_2022])

resultados['votos'] = resultados['votos'].astype('int')
resultados['votos_totais'] = resultados['votos_totais'].astype('int')
resultados['votos_validos'] = resultados['votos_validos'].astype('int')
resultados['turno'] = resultados['turno'].astype('int')
resultados['ano'] = resultados['ano'].astype('int')

In [393]:
pesquisas = pd.read_csv('pesquisas_eleitorais.csv')
aux_idpoder360 = pesquisas[['nome_candidato','id_candidato_poder360']].drop_duplicates()
resultados = pd.merge(resultados,aux_idpoder360,left_on='nomes',right_on='nome_candidato').drop('nome_candidato',1)

resultados = resultados.rename(columns={'nomes':'nome','votos_totais':'total_votos','id_candidato_poder360':'id_poder_360','estado':'local_que_concorreu'})
resultados = resultados[['nome','partido','votos','votos_validos','total_votos','turno','ano','id_poder_360','local_que_concorreu']]
resultados

/var/folders/ts/qnrqfkqx7ldcn69lydmzhd2h0000gn/T/ipykernel_21550/2739623123.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  resultados = pd.merge(resultados,aux_idpoder360,left_on='nomes',right_on='nome_candidato').drop('nome_candidato',1)


,nome,partido,votos,votos_validos,total_votos,turno,ano,id_poder_360,local_que_concorreu
0,Gladson Cameli,PP,223993,443429,443429,1,2018,4820,ac
1,Gladson Cameli,PROGRESSISTAS,242100,455438,455438,1,2022,4820,ac
2,Marcus Alexandre,PT,144071,443429,443429,1,2018,3578,ac
3,Coronel Ulysses,PSL,45032,443429,443429,1,2018,4821,ac
4,David Hall,AVANTE,1215,443429,443429,1,2018,4825,ac
...,...,...,...,...,...,...,...,...,...
261,Izalci,PSDB,70584,1807484,1807484,1,2022,695,df
262,Keka Bagno,PSOL,13613,1807484,1807484,1,2022,6184,df
263,Lucas Salles,DC,4218,1807484,1807484,1,2022,6205,df
264,Robson,PSTU,841,1807484,1807484,1,2022,1447,df


In [394]:
resultados.to_csv('resultados_governador.csv',index=False)

In [ ]:
# colocar na analise de ml (verificar as info que tinha na tabela de resultado e replicar) 
# colocar na analise de ml (verificar passo a passo pra ver se n tem nenum outro problema!)